# Text Encoder: Roberta
> A simple intro and explaination on the backbone model

Jie Wang


Reference: https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb

TODO 
1. 完善这个demo(notebook), 明天可以直接跑 
2. 介绍一下text2text model 
overall family 
3. 介绍一下roberta 是怎么在transmot.py 文件里使用的
https://huggingface.co/docs/transformers/model_doc/roberta
4. 分析一下我们可以换成哪些其他的text encoder ( 从hugging face的transformers family 选一个更强的)
> For text-to-text work, you would typically use models like RoBERTa, BERT, GPT-3, or T5, which are designed to understand and generate human language. These models are trained on large text corpora and are better suited for language-only tasks.

假设更改text encoder, 我们需要做哪些改变？> GPT 分析一下
> Linguistic Extractor. Next, we study different linguistic extractors, including the widely-used Transformer-based text encoders (e.g., BERT [6] and Distill-BERT [37]) and the simple word embedding methods (e.g., Glove [33] and FastText [31]). As shown in Table 3 (c), these Transformerbased encoders achieve comparable performance in comparison to the RoBERTa [25], while the simple embedding methods are insufficient in our cross-modal learning
5. CLIP : RJJ 负责， 是一个image2text model , 用于指示roberta的query 
6. 

Model Details. We adopt visual backbone ResNet-50 [10]
and text encoder RoBERTa [25] in our TransRMOT. Similar to Deformable DETR [62], the last three stage features
{I
3
t
, I
4
t
, I
5
t } from the visual backbone are used for further
cross-modal learning. Besides, the lowest resolution feature
map I
6
t
is added via a 3×3 convolution with spatial stride
2 on the I
5
t
. Each of the multi-scale feature maps is independently performed the cross-modal fusion. After that,
deformable attention in the encoder and decoder integrates
the multi-scale features. The architecture and number of the
encoder and decoder layer follow the setting of [62]. The
number of detect query is set as N = 300.

Roberta（Robustly Optimized BERT Pretraining Approach）是一种基于BERT（Bidirectional Encoder Representations from Transformers）的改进模型。它通过在更大的数据集上进行更长时间的训练，以及移除BERT中的Next Sentence Prediction（NSP）任务，实现了对语言模型性能的显著提升。Roberta在多项自然语言处理任务中表现出色，特别是在文本分类、情感分析和问答等方面。

### 使用Hugging Face的Roberta模型

Hugging Face提供了一个非常方便的接口来使用Roberta模型。以下是使用Roberta的基本步骤，包括加载模型、分词器，以及如何使用模型进行预测。

#### 安装Hugging Face库

首先，需要安装Hugging Face的`transformers`库。这可以通过以下命令完成：

```python
    !pip install transformers
```




其次，为了在国内使用hugging face的weight, 我们使用了如下的学术加速脚本：
- 来自AutoDL server documentation

In [10]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [11]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-base')
unmasker("Hello I'm a <mask> model.")


[{'sequence': "Hello I'm a male model.",
  'score': 0.33065351843833923,
  'token': 2943,
  'token_str': ' male'},
 {'sequence': "Hello I'm a female model.",
  'score': 0.04655401408672333,
  'token': 2182,
  'token_str': ' female'},
 {'sequence': "Hello I'm a professional model.",
  'score': 0.04232970252633095,
  'token': 2038,
  'token_str': ' professional'},
 {'sequence': "Hello I'm a fashion model.",
  'score': 0.03721661865711212,
  'token': 2734,
  'token_str': ' fashion'},
 {'sequence': "Hello I'm a Russian model.",
  'score': 0.032536473125219345,
  'token': 1083,
  'token_str': ' Russian'}]

In [6]:
#### 加载Roberta模型和分词器

# 接下来，从Hugging Face库中加载Roberta模型和对应的分词器：

from transformers import RobertaModel, RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

#### 使用Roberta进行文本编码

# 以下是如何将文本编码为特征向量的示例：

# 示例文本
text = "Hello, world! This is a test for Roberta."

# 使用分词器进行编码
inputs = tokenizer(text, return_tensors="pt")

# 获取编码的输出
outputs = model(**inputs)

# 最后一层的隐藏状态
last_hidden_states = outputs.last_hidden_state
last_hidden_states

tensor([[[-0.1064,  0.1032, -0.0107,  ..., -0.0403, -0.0634, -0.0219],
         [-0.1265,  0.1128, -0.0229,  ...,  0.0068,  0.0624, -0.0472],
         [ 0.0898,  0.0612,  0.0918,  ..., -0.2020,  0.0482,  0.0777],
         ...,
         [ 0.0380,  0.1408, -0.0050,  ..., -0.2869, -0.0685,  0.0693],
         [-0.0968,  0.0959, -0.0369,  ..., -0.0635, -0.0698, -0.0520],
         [ 0.0353,  0.0537,  0.1590,  ...,  0.2409, -0.0235, -0.0134]]],
       grad_fn=<NativeLayerNormBackward>)

In [7]:

#### 使用Roberta进行文本分类

# 如果你想将Roberta用于文本分类，你需要在Roberta的基础上添加一个分类头。以下是一个示例：

from transformers import RobertaForSequenceClassification
import torch

# 加载用于序列分类的Roberta模型
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# 准备输入数据
inputs = tokenizer("This is a sample text for classification.", return_tensors="pt")

# 预测
outputs = model(**inputs)

# 获取预测结果
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

'\n- 当使用模型进行预测时，确保输入的文本被正确地分词。\n- 对于不同的任务（如文本分类、问答等），可能需要使用特定于任务的模型版本（如`RobertaForSequenceClassification`、`RobertaForQuestionAnswering`等）。\n\n这些脚本为基本的使用示例，针对具体任务可能需要进行适当的修改。\n'

In [8]:
predictions

tensor([[0.4576, 0.5424]], grad_fn=<SoftmaxBackward>)

#### 注意事项
- 当使用模型进行预测时，确保输入的文本被正确地分词。
- 对于不同的任务（如文本分类、问答等），可能需要使用特定于任务的模型版本（如`RobertaForSequenceClassification`、`RobertaForQuestionAnswering`等）。
